# Librerías

In [1]:
import itertools
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)#Prueba evaluación y entrenamiento un solo individuo. 
#Vectores completos
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import random

# Databases

In [454]:
#Reading all the cvs
#Nss learning
nsl_training=pd.read_csv('./Final_Datasets/MINMAX_FD_NSL_Tra.csv')
#Nsl testing
nsl_testing=pd.read_csv('./Final_Datasets/MINMAX_FD_NSL_Test.csv')

# Labels

In [6]:
#NSL TESTING LABELS

#Index binary (2 clases)
L_nsl_testing_indexbinary=pd.read_csv("./Final_Datasets/L_nsl_testing_indexbinary.csv")
#Index by Group (5 clases)
L_nsl_testing_indexbygroup=pd.read_csv("./Final_Datasets/L_nsl_testing_indexbygroup.csv")
#Index tipe of attack (42? clases)
L_nsl_testing_indextipeofattack=pd.read_csv("./Final_Datasets/L_nsl_testing_indextipeofattack.csv")


#nsl TRAINING LABELS
# 2 clases
L_nsl_training_indexbinary=pd.read_csv("./Final_Datasets/L_nsl_training_indexbinary.csv")
#Index by Group (5 clases)
L_nsl_training_indexbygroup=pd.read_csv("./Final_Datasets/L_nsl_training_indexbygroup.csv")
#Index tipe of attack (42? clases)
L_nsl_training_indextipeofattack=pd.read_csv("./Final_Datasets/L_nsl_training_indextipeofattack.csv")

# Conversión de Pandas->Numpy (array) not Matrix

In [455]:
n_nsl_training=nsl_training.as_matrix(columns=None)
n_nsl_testing=nsl_testing.as_matrix(columns=None)

# Lectura del modelo

In [4]:
from sklearn.ensemble import RandomForestClassifier

modelo=RandomForestClassifier(criterion='entropy',n_jobs=2, 
                              bootstrap=True, max_depth=10, max_features='sqrt', min_samples_leaf=2, 
                              min_samples_split=100, n_estimators=8)


# Creacion de las funciones a utilizar

In [10]:
def muta_vector_binario(dimensiones_temporales,M=40):
    """Dimensiones temporales: vector binario que contiene las dimensiones actuales de la ultima
        iteración probada.
        M=cantidad total de dimensiones a mutar. Si no se otorga un número
        M es igual a la máxima cantidad de dimensiones en NSL-KDD"""
    #Genera tres números aleatorios entre 0 y M
    #Dichos numeros aleatorios seran los indices a mutar entre de (dimensiones_temporales)
    indice=sorted(random.sample(list(range(0,dimensiones_temporales.shape[1])), M))
    #Aplica la operación negación binaria a los valores del indice
    for numero_indice in indice:
        if dimensiones_temporales[indice]==0:
            dimensiones_temporales[indice]=1
        else:
            dimensiones_temporales[indice]=0
    #regresa el vector mutado
    vector_mutado=dimensiones_temporales
    return vector_mutado
    

In [456]:
#inicio RMHC
#M, comienza con la máxima cantidad de dimensiones posibles
M=n_nsl_training.shape[1]


#declaración de la primea solución del espacio de búsqueda. 
#de forma completamente aleatoria
#La primer solución se considera la mejor en la primer iteración
dimensiones_mejor=np.random.randint(2, size=M)

#se filtra la base de datos con las dimensiones de dimensiones mejor
#SE hacen cero las dimensiones a omitir
#dimensiones_mejor contiene la mejor iteración, entonces es la dimensión activa
training=n_nsl_training*dimensiones_mejor
testing=n_nsl_testing*dimensiones_mejor

#Se buscan esos ceros en el indice (dimensiones suspendidas)
zeros=np.where(dimensiones_mejor == 0)

#se obtienen los indices del los unos (dimensiones activas)
unos=np.where(dimensiones_mejor==1)


#se eliminan esas dimensiones
training=np.delete(training,zeros,axis=1)
testing=np.delete(testing,zeros,axis=1)


#se declara un vector que contiene todas las dimensiones en cada iteración
dimensiones_iteracion=[]
dimensiones_iteracion.append(dimensiones_mejor)



#se entrena el modelo
modelo.fit(training,L_nsl_training_indexbinary.values.ravel())
#se obtienen las predicciones
predicciones=modelo.predict(testing)

#se obtiene el accuracy score
score_mejor=accuracy_score(L_nsl_testing_indexbinary,predicciones)
#se declara un vector que contiene todos los score de todas las iteraciones

score_iteracion=[]
score_iteracion.append(score_mejor)


print("Dimensiones activas: ")
print(unos)
print("Total de dimensiones activas: %s" % np.array(unos).shape[1])
print("Porcentaje de exactitud: %s (accuracy score)" %accuracy_score(L_nsl_testing_indexbinary,predicciones))

Dimensiones activas: 
(array([ 0,  7,  8,  9, 11, 13, 14, 15, 16, 17, 21, 23, 25, 26, 33, 35, 36,
       39]),)
Total de dimensiones activas: 18
Porcentaje de exactitud: 0.724659539547 (accuracy score)


In [457]:
n_nsl_testing.shape

(22543, 40)

In [458]:
n_nsl_training.shape

(125973, 40)

In [459]:
training.shape

(125973, 18)

In [460]:
testing.shape

(22543, 18)

In [11]:
def obtencion_parametros_binarios(vector_actual, M=40):
    """M=máximo número de dimensiones a mutar
    vector_actual= vector binario, relacionado con la ultima iteracion
    puede o no ser el mejor.
    Regresa los zeros y unos actuales,
    el ultimo vector binario,
    los dataset filtrados"""

    #Agregar la función para calcular M

    
    #Muta el ultimo vector
    dimensiones_temporales=muta_vector_binario(vector_actual,M)

    #se filtra la base de datos con las dimensiones de dimensiones mejor
    #SE hacen cero las dimensiones a omitir
    #dimensiones_temporales contiene la mejor iteración, entonces es la dimensión activa
    training=nsl_training*dimensiones_temporales
    testing=nsl_testing*dimensiones_temporales

    #Se buscan esos ceros en el indice (dimensiones suspendidas)
    zeros=np.where(dimensiones_temporales == 0)

    #se obtienen los indices del los unos (dimensiones activas)
    unos=np.where(dimensiones_temporales==1)

    #se eliminan esas dimensiones
    training=np.delete(training,zeros,axis=1)
    testing=np.delete(testing,zeros,axis=1)

    
    return [dimensiones_temporales,zeros,unos,training,testing]

# Inicio del algoritmo RMHC

In [12]:
from sklearn.ensemble import RandomForestClassifier

modelo=RandomForestClassifier(criterion='entropy',n_jobs=2, 
                              bootstrap=True, max_depth=10, max_features='sqrt', min_samples_leaf=2, 
                              min_samples_split=100, n_estimators=8)


In [15]:
i=0
M=40
iteraciones_maximas=10
mejor_solucion=np.random.randint(2, size=M)
#almacena los score de todas las iteraciones
registro_score_iteracion=[]
#almacena el vector binario de todas las iteraciones
registro_dimensiones_iteracion=[]
#almacena unicamente el score de los vectores binarios aceptados
registro_score_valores_aceptados=[]
#almacena el vector binario de los valores aceptados
registro_dimensiones_valores_aceptados=[]
#almacena el indice de las dimensiones activas de los valores aceptados
registro_unos_valores_aceptados=[]

mejor_solucion

while (i<iteraciones_maximas):
    [dimensiones_temporales,zeros,unos,training,testing]=obtencion_parametros_binarios(mejor_solucion)    
    #Se evaluan los resultados
    #se entrena el modelo
    modelo.fit(training,L_nsl_training_indexbinary.values.ravel())
    #se obtienen las predicciones
    predicciones=modelo.predict(testing)

    #se obtiene el accuracy score
    score_actual=accuracy_score(L_nsl_testing_indexbinary,predicciones)
    #se declara un vector que contiene todos los score de todas las iteraciones

    #Se almacenan los resultados
    registro_score_iteracion.append(score_actual)
    registro_dimensiones_iteracion.append(dimensiones_temporales)
    

    
    #Se imprimen los resultados
    print("Dimensiones activas: ")
    print(unos)
    print("Total de dimensiones activas: %s" % np.array(unos).shape[1])
    print("Porcentaje de exactitud: %s (accuracy score)" %accuracy_score(L_nsl_testing_indexbinary,predicciones))
    
    
    i=i+1
    

IndexError: tuple index out of range

In [41]:
mejor_solucion=np.random.randint(2, size=M)
mejor_solucion

array([0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1])

In [ ]:
#evaluación del siguiente vector


    #se eliminan esas dimensiones
    training=np.delete(training,zeros,axis=1)
    testing=np.delete(testing,zeros,axis=1)


    #se declara un vector que contiene todas las dimensiones en cada iteración
    dimensiones_iteracion=[]
    dimensiones_iteracion.append(dimensiones_mejor)



    #se entrena el modelo
    modelo.fit(training,L_nsl_training_indexbinary.values.ravel())
    #se obtienen las predicciones
    predicciones=modelo.predict(testing)

    #se obtiene el accuracy score
    score_mejor=accuracy_score(L_nsl_testing_indexbinary,predicciones)
    #se declara un vector que contiene todos los score de todas las iteraciones

    score_iteracion=[]
    score_iteracion.append(score_mejor)


    print("Dimensiones activas: ")
    print(unos)
    print("Total de dimensiones activas: %s" % np.array(unos).shape[1])
    print("Porcentaje de exactitud: %s (accuracy score)" %accuracy_score(L_nsl_testing_indexbinary,predicciones))

In [433]:
print(mejor_solucion)

[1 1 0 1 1 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 0 1 0 1 1 1 1 1 0 0 0 1 0 1 0 1 0
 1 0 0]


In [444]:
def muta_vector_binario(dimensiones_temporales,M=20):
    """Dimensiones temporales: vector binario que contiene las dimensiones actuales de la ultima
        iteración probada.
        M=cantidad total de dimensiones a mutar. Si no se otorga un número
        M es igual a la máxima cantidad de dimensiones en NSL-KDD"""
    
    vector_mutado=np.array(dimensiones_temporales)
    #Genera tres números aleatorios entre 0 y M
    #Dichos numeros aleatorios seran los indices a mutar entre de (vector_mutado)
    indice=sorted(random.sample(list(range(0,vector_mutado.shape[0])), M))
    print(indice)
    #Aplica la operación negación binaria a los valores del indice
    for numero_indice in indice:
        if vector_mutado[numero_indice]==0:
            vector_mutado[numero_indice]=1
        else:
            vector_mutado[numero_indice]=0
    #regresa el vector mutado
    return vector_mutado

In [451]:
print(mejor_solucion)
print("\n")
test=muta_vector_binario(mejor_solucion)
print(test)

[1 1 0 1 1 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 0 1 0 1 1 1 1 1 0 0 0 1 0 1 0 1 0
 1 0 0]


[5, 6, 7, 10, 13, 14, 16, 19, 20, 21, 22, 24, 25, 26, 28, 29, 31, 36, 38, 39]
[1 1 0 1 1 1 1 1 0 1 0 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 1 1 1 0 0 0 1 0 1 1
 1 1 1]


In [45]:
mejor_solucion.shape[0]

40

In [163]:
dimensiones_temporales=mejor_solucion

In [162]:
indice=sorted(random.sample(list(range(0,dimensiones_temporales.shape[0])), 20))
indice

[1, 8, 9, 11, 13, 15, 16, 17, 18, 20, 23, 25, 26, 28, 30, 31, 32, 34, 35, 39]

In [165]:
dimensiones_temporales[0]

0